In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pandas as pd
import clip
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch.nn as nn

base_dir = "/content/drive/MyDrive/biodata Project/MRNet-v1.0"
plane = "sagittal"  # can be 'axial', 'coronal', 'sagittal'
label_csv = os.path.join(base_dir, "train-abnormal.csv")
image_dir = os.path.join(base_dir, "train", plane)


# Load labels
labels_df = pd.read_csv(label_csv)
#print(f"Total scans: {len(labels_df)}")
# Choose a sample (change index to view other examples)
sample_index = 0
img_id = str(labels_df.loc[sample_index, '1']).zfill(4)
label = labels_df.loc[sample_index, '1']

# Load image
img_path = os.path.join(image_dir, f"{img_id}.npy")
scan = np.load(img_path)

print(f"Scan shape: {scan.shape} — Label: {label}")

# Plot a few slices
num_slices = scan.shape[0]
slices_to_plot = np.linspace(0, num_slices - 1, 6, dtype=int)

plt.figure(figsize=(12, 6))
for i, slice_idx in enumerate(slices_to_plot):
    plt.subplot(2, 3, i+1)
    plt.imshow(scan[slice_idx], cmap='gray')
    plt.title(f"Slice {slice_idx}")
    plt.axis('off')

plt.suptitle(f"Sample {img_id} — Abnormal: {label}")
plt.tight_layout()
plt.show()


abnormal_df = pd.read_csv(os.path.join(base_dir, "train-abnormal.csv"))
acl_df = pd.read_csv(os.path.join(base_dir, "train-acl.csv"))
meniscus_df = pd.read_csv(os.path.join(base_dir, "train-meniscus.csv"))

abnormal_df.columns = ['exam', 'abnormal']
acl_df.columns = ['exam', 'acl']
meniscus_df.columns = ['exam', 'meniscus']

merged_df = abnormal_df.merge(acl_df, on='exam').merge(meniscus_df, on='exam')


def generate_caption(row):
    if row['abnormal'] == 0:
        return "The depicted knee appears to be normal."

    findings = []
    if row['acl'] == 1:
        findings.append("an ACL tear")
    if row['meniscus'] == 1:
        findings.append("a meniscus tear")

    if findings:
        return "The depicted knee has " + " and ".join(findings) + "."
    else:
        return "The depicted knee has an unspecified abnormality that is neither an acl nor a meniscus tear."


merged_df['caption'] = merged_df.apply(generate_caption, axis=1)
merged_df

merged_df["caption"].unique()

### Data Preprocessing

# Use CLIP preprocessing (from OpenAI or OpenCLIP)
clip_preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073),
                         std=(0.26862954, 0.26130258, 0.27577711))
])

def load_exam_mri(path):
    scan = np.load(path)  # shape: (slices, H, W)

    # Choose middle 3 slices and stack to simulate RGB
    mid = scan.shape[0] // 2
    slices = scan[mid - 1: mid + 2]

    # Normalize to [0, 255] and convert to uint8 for PIL
    slices = np.stack([((s - s.min()) / (s.max() - s.min()) * 255).astype(np.uint8) for s in slices], axis=-1)

    # Convert to PIL and preprocess
    img = Image.fromarray(slices)
    return clip_preprocess(img)


device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# We will only use model.visual (vision encoder)


def encode_image(tensor_image):
    tensor_image = tensor_image.unsqueeze(0).to(device)  # add batch dim
    with torch.no_grad():
        image_embedding = model.encode_image(tensor_image)
    return image_embedding  # shape: (1, 512)


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

gpt2 = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

# Freeze CLIP if you want:
for p in model.visual.parameters():
    p.requires_grad = False


class ClipCaptionModel(nn.Module):
    def __init__(self, clip_dim=512, prefix_len=10):
        super().__init__()
        self.gpt = GPT2LMHeadModel.from_pretrained("gpt2")
        self.prefix_len = prefix_len
        self.clip_project = nn.Linear(clip_dim, self.gpt.config.n_embd * prefix_len)

    def forward(self, image_embedding, captions, attention_mask):
        prefix_embedding = self.clip_project(image_embedding).view(-1, self.prefix_len, self.gpt.config.n_embd)
        caption_embeddings = self.gpt.transformer.wte(captions)

        # Concatenate projected image embedding with caption tokens
        embeddings = torch.cat((prefix_embedding, caption_embeddings), dim=1)

        # Shift attention mask accordingly
        extended_attention = torch.cat((
            torch.ones((captions.shape[0], self.prefix_len), device=attention_mask.device),
            attention_mask
        ), dim=1)

        outputs = self.gpt(inputs_embeds=embeddings, attention_mask=extended_attention, labels=captions)
        return outputs


optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

for batch in dataloader:
    images, captions = batch  # preprocessed tensors
    image_embeddings = model.encode_image(images)

    tokens = tokenizer(captions, return_tensors="pt", padding=True, truncation=True)
    input_ids = tokens.input_ids.to(device)
    attention_mask = tokens.attention_mask.to(device)

    loss = clip_gpt_model(image_embeddings, input_ids, attention_mask).loss

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
